In [134]:
import pandas as pd
import numpy as np
from ast import literal_eval
import math

In [35]:
from tqdm import tqdm
from gensim.models import Doc2Vec
from gensim.models.doc2vec import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
from gensim.models.doc2vec import TaggedDocument

C:\Users\morad\Anaconda3\lib\site-packages\tqdm\std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## load datasets after cleaning

In [75]:
single_labels = pd.read_csv('..\git_data_single_label.csv')

In [76]:
dev2vec_repos_dt = pd.read_csv(r'..\dev2vec_repo_clean.csv')

In [104]:
dev2vec_repos_dt.text_all = dev2vec_repos_dt.text_all.apply(literal_eval)

In [107]:
#dev2vec_repos_dt['text_all'][0]

In [77]:
dev2vec_issues_dt = pd.read_csv(r'..\dev2vec_issue_clean.csv')

In [108]:
dev2vec_issues_dt.text_all = dev2vec_issues_dt.text_all.apply(literal_eval)

In [109]:
trainset, testset = train_test_split(single_labels, test_size=0.1, random_state=45)

In [110]:
validation, testset = train_test_split(testset, test_size=0.5, random_state=42)

In [111]:
trainset_dev = list(trainset['gh_login'])

In [112]:
testset_dev = list(testset['gh_login'])

In [113]:
validation_dev = list(validation['gh_login'])

In [114]:
trainset_dev2vec_repo= dev2vec_repos_dt[dev2vec_repos_dt['tags'].isin(trainset_dev)]

In [116]:
testset_dev2vec_repo= dev2vec_repos_dt[dev2vec_repos_dt['tags'].isin(testset_dev)]

In [117]:
val_dev2vec_repo= dev2vec_repos_dt[dev2vec_repos_dt['tags'].isin(validation_dev)]

In [118]:
trainset_dev2vec_issue= dev2vec_issues_dt[dev2vec_issues_dt['tags'].isin(trainset_dev)]

In [119]:
testset_dev2vec_issue= dev2vec_issues_dt[dev2vec_issues_dt['tags'].isin(testset_dev)]

In [120]:
val_dev2vec_issue= dev2vec_issues_dt[dev2vec_issues_dt['tags'].isin(validation_dev)]

### Train dev2vec:repos

In [123]:
train = trainset_dev2vec_repo.apply(
    lambda r: TaggedDocument(words=r['text_all'], tags=[r.tags]), axis=1)

In [124]:
test = testset_dev2vec_repo.apply(
    lambda r: TaggedDocument(words=r['text_all'], tags=[r.tags]), axis=1)

In [67]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [96]:
#Define the Cosine Similarity Function
def cos_sim(av,bv):
    return(sum(av*bv)/math.sqrt(sum(av*av)*sum(bv*bv)))

In [97]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    model.random.seed(0)
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, epochs=25)) for doc in sents])
    return targets, regressors

In [ ]:
%%time
dev2vec_repos_dmm.train(train.values, total_examples=len(train.values), epochs=dev2vec_repos_dmm.epochs)

In [ ]:
%%time
dev2vec_repos_dmm = Doc2Vec(dm=1, vector_size=230, window=5, negative=5, min_count=5,epochs=15)
dev2vec_repos_dmm.build_vocab(train.values)

In [146]:
#dev2vec_repos_dmm.save("dev2vec_repos", sep_limit=25000)

In [127]:
%%time
y_train, X_train = vec_for_learning(dev2vec_repos_dmm, train)

Wall time: 4min 14s


In [129]:
y_test, X_test = vec_for_learning(dev2vec_repos_dmm, test)

In [130]:
Y_train,labels=zip(*[(label,label) for label in trainset_dev2vec_repo['label']] )
Y_test,labels=zip(*[(label,label) for label in testset_dev2vec_repo['label']] )

In [131]:
u_id_lst=[]
for row in train.values:
    u_id_lst.append(row.tags[0])

In [132]:
sim_train=pd.DataFrame(columns=['u_id','cos_sim'], dtype=object)

In [135]:
for uid in range(0,len(u_id_lst)):
    vec_train=X_train[uid]
    username=u_id_lst[uid]
    vec_user = dev2vec_repos_dmm.docvecs[username]
    sim_train=sim_train.append({'u_id':username,'cos_sim':cos_sim(vec_train,vec_user)},ignore_index=True)

In [136]:
(len(sim_train)-len(sim_train[sim_train['cos_sim']<0.85]))/len(sim_train)

0.9961270333075135

In [95]:
pd.DataFrame(Y_test)[0].value_counts()

Frontend         57
Mobile           32
Backend          19
DataScientist    18
DevOps           10
Name: 0, dtype: int64

_________________________________________________________________________________________________________________

X_train and X_test are the vector of developers in train and test sets. Y_train and Y_test are the associated labels. Vectors are the embedding reprsentation of the expertise of developers and can be fed into any classifiers or any other downstream task.

-------------------------------------------------------------------------------------------------------------
## Train dev2vec:issues

In [160]:
train = trainset_dev2vec_issue.apply(
    lambda r: TaggedDocument(words=r['text_all'], tags=[r.tags]), axis=1)

In [161]:
test = testset_dev2vec_issue.apply(
    lambda r: TaggedDocument(words=r['text_all'], tags=[r.tags]), axis=1)

In [ ]:
%%time
dev2vec_issues_dmm = Doc2Vec(dm=1, vector_size=150, window=5, negative=5, min_count=5,epochs=25)
dev2vec_issues_dmm.build_vocab(train.values)

In [ ]:
%%time
dev2vec_issues_dmm.train(train.values, total_examples=len(train.values), epochs=dev2vec_issues_dmm.epochs)

In [202]:
dev2vec_issues_dmm.save("dev2vec_issues", sep_limit=10000)

In [192]:
%%time
y_train_issue, X_train_issue = vec_for_learning(dev2vec_issues_dmm, train)

Wall time: 27.9 s


In [193]:
y_test_issue , X_test_issue = vec_for_learning(dev2vec_issues_dmm, test)

In [194]:
Y_train,labels=zip(*[(label,label) for label in dev2vec_issues_dmm['label']] )
Y_test,labels=zip(*[(label,label) for label in dev2vec_issues_dmm['label']] )

In [195]:
u_id_lst=[]
for row in train.values:
    u_id_lst.append(row.tags[0])

In [196]:
sim_train=pd.DataFrame(columns=['u_id','cos_sim'], dtype=object)

In [197]:
for uid in range(0,len(u_id_lst)):
    vec_train=X_train_issue[uid]
    username=u_id_lst[uid]
    vec_user = dev2vec_issues_dmm.docvecs[username]
    sim_train=sim_train.append({'u_id':username,'cos_sim':cos_sim(vec_train,vec_user)},ignore_index=True)

In [198]:
(len(sim_train)-len(sim_train[sim_train['cos_sim']<0.85]))/len(sim_train)

0.9668367346938775